# Notes

This assignment is devoted to `pandas`. It covers indexing and filtering, and some `groupby` and `join` operations. The assignment roughly corresponds to Week 4 and the beginning of Week 5 of the course.

The main dataset you'll be using is [Titanic](https://www.kaggle.com/c/titanic).

In [1]:
%pylab inline
plt.style.use("bmh")

ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
plt.rcParams["figure.figsize"] = (6,6)

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
STUDENT = "Adam Cohn"
ASSIGNMENT = 4
TEST = False

In [ ]:
if TEST:
    import solutions
    total_grade = 0
    MAX_POINTS = 16

# Indexing and filtering

### 1. Fixing age (1 point).

There are several known mistakes in the Titanic dataset.

Namely, [Julia Florence Siegel](https://www.encyclopedia-titanica.org/titanic-survivor/julia-florence-cavendish.html) (Mrs. Tyrell William Cavendish) is mistakenly marked as being 76 years old (the age she actually died, but many years after Titanic).

You must replace age value for her with her actual age at the time (25) and return the dataset. Input is indexed with `PassengerId` and is a concatenation of train and test sets. You must return a copy of the dataframe, and not perform replacement in original dataframe. Structure and indexing must be the same as in input.

In [ ]:
df = pd.read_csv('titanic.csv', index_col='PassengerId')

In [ ]:
def fix_age(df):
    """Fix age for Julia Florence Siegel."""
    data = df.copy()
    data.loc[data.index == 988, 'Age'] = 25
    return data

In [ ]:
PROBLEM_ID = 1

if TEST:
    total_grade += solutions.check(STUDENT, PROBLEM_ID, fix_age)

In [ ]:
data = fix_age(df)
# df.loc[df.index == 988].head()
data.loc[data.index == 988].head()

### 2. Embarkment port distribution (1 point).

You must find the value counts for embarkment port (`Embarked` column) for the passengers, who travelled in 3-d class, were male and between 20 and 30 years old (both inclusive). No need to treat missing values separately.

Input is indexed with `PassengerId` and is a concatenation of train and test sets. You must return series, indexed with values from `Embarked`, according to `.value_counts()` method semantics.

In [ ]:
def embarked_stats(df):
    """Calculate embarkment port statistics."""
    return df.loc[(df.Pclass == 3) & (df.Sex == 'male') & 
                      (df.Age.between(20, 30, inclusive=True))].Embarked.value_counts()

In [ ]:
PROBLEM_ID = 2

if TEST:
    total_grade += solutions.check(STUDENT, PROBLEM_ID, embarked_stats)

In [ ]:
embarked = embarked_stats(data)
embarked

### 3. Fill missing age values (1 point).

Some age values are missing in the Titanic dataset. You need to calculate average age, and fill missing age values in `Age` column.

Input is indexed with `PassengerId` and is a concatenation of train and test sets. Output must be a **new** dataframe with the same structure, but without missing values in `Age` column.

In [ ]:
def fix_age(df):
    """Fix missing age values."""
    data = df.copy()
    data['Age'] = data.Age.fillna(data.Age.mean(skipna=True))
    return data

In [ ]:
PROBLEM_ID = 3

if TEST:
    total_grade += solutions.check(STUDENT, PROBLEM_ID, fix_age)

In [ ]:
data = fix_age(data)
data.Age.head(20)

### 4. Child travelling alone (1 point).

You must find a child (`Age<10`) on-board, who was travelling without siblings or parents and find a name of her nursemaid.

Input is indexed with `PassengerId` and is a concatenation of train and test sets. Output must be a tuple of two strings, collected from `Name` column, with one being child's name and second being nursemaid's name. It's known, that there's only one child like this.

In [ ]:
def get_nursemaid(df):
    child =  df[(df["Age"] < 10) & (df["SibSp"] == 0) & (df["Parch"] == 0)]["Ticket"]
    names = df.loc[(df["Ticket"] == child.iloc[0])].sort_values(by='Age', ascending=True)['Name']
    return tuple(names)

In [ ]:
PROBLEM_ID = 4

if TEST:
    total_grade += solutions.check(STUDENT, PROBLEM_ID, get_nursemaid)

In [ ]:
nurse = get_nursemaid(data)
nurse

### 5. Port with the most children embarked (1 point).

You must find, which port had the largest percentage of children (`Age<10`) embarked, i.e. number of children divided by total number of passengers embarked.

Input is indexed with `PassengerId` and is a concatenation of train and test sets. Output must be a single string with port letter.

In [ ]:
def get_port(df):
    """Get port with the most children embarked."""
#     data = df.copy()
#     data['is_child'] = 0
#     data.loc[data.Age < 10, 'is_child'] = 1
#     data = data.groupby('Embarked').is_child.value_counts(normalize=True)
#     return data.loc[data.index.get_level_values('is_child') == 1].nlargest(1).index[0][0]
    return (df.loc[df.Age < 10].groupby('Embarked').size()/df.groupby('Embarked').size()).idxmax()

In [ ]:
PROBLEM_ID = 5

if TEST:
    total_grade += solutions.check(STUDENT, PROBLEM_ID, get_port)

In [ ]:
port = get_port(data)
port

### 6. Passengers per ticket (2 points).

Calculate average and maximum number of passengers per ticket.

Input is indexed with `PassengerId` and is a concatenation of train and test sets. Output must be a tuple of two values.

In [ ]:
def get_ticket_stats(df):
    """Calculate passenger per ticket statistics."""
    return tuple(df.groupby('Ticket')['Name'].count().aggregate(['mean','max']))

In [ ]:
PROBLEM_ID = 6

if TEST:
    total_grade += solutions.check(STUDENT, PROBLEM_ID, get_ticket_stats)

In [ ]:
stats = get_ticket_stats(data)
stats

### 7. Fare per passenger (3 points).

For each individual ticket, you must calculate fare per person for that ticket, and then calculate averages for each class. Note, that you will need to apply `groupby` and then you may consider using `.first()` of resulting `DataFrameGroupBy`.

Input is indexed with `PassengerId` and is a concatenation of train and test sets. Output must be `Series` with three elements, indexed by class.

In [ ]:
def get_fare_per_pass(df):
    """Calculate fare per passenger for different classes."""
    data = df.copy()
    data['fare_person'] = data['Fare'] / data.groupby('Ticket').Ticket.transform('count')
    return data.groupby('Pclass')['fare_person'].mean()

In [ ]:
PROBLEM_ID = 7

if TEST:
    total_grade += solutions.check(STUDENT, PROBLEM_ID, get_fare_per_pass)

In [ ]:
fare = get_fare_per_pass(data)
fare.head()

In [ ]:
data.groupby('Ticket').Ticket.transform('count')

In [ ]:
data['Fare']

In [ ]:
data['Fare'] / data.groupby('Ticket').Ticket.transform('count')

### 8. Fill missing age values (3 points).

In problem 3 you filled missing age values with average for all passengers. Now, you need to fill them according to class and sex. For example, for a female passenger from 2d class, missing age value must be filled with average age of females in 2d class.

In this problem, you may need joins and `.apply()`, although there are several ways to get the same result.

Input is indexed with `PassengerId` and is a concatenation of train and test sets. Output must be a **new** dataframe with the same structure as input, but without missing values in `Age` column.

In [ ]:
def fix_age_groupped(df):
    """Fill missing age values."""
    data = df.copy()
    meanAge = data.groupby(['Pclass','Sex'])['Age'].mean()
    data = data.reset_index().set_index(['Pclass','Sex'])
    data['Age'] = data['Age'].fillna(meanAge)
    data = data.reset_index()
    data.index = df.index
    return data[df.columns]

def fix_age_groupped2(df):
    cols = df.columns.tolist()
    age_group = df.groupby(['Pclass','Sex'])['Age'].mean()
    new_df = pd.merge(df, age_group, how="inner", left_on=['Pclass','Sex'], right_on=['Pclass','Sex'], suffixes=('','_new'))
    new_df["Age"].fillna(new_df["Age_new"], inplace=True)
    return new_df

In [ ]:
PROBLEM_ID = 8

if TEST:
    total_grade += solutions.check(STUDENT, PROBLEM_ID, fix_age_groupped)

In [ ]:
data = fix_age_groupped(df)
data2 = fix_age_groupped2(df)
data.loc[df.Age.isna()]

In [ ]:
data2.loc[df.Age.isna()]

### 9. Finding couples (3 points).

Based on the code from Lecture 5, build a dataframe of couples. Filter it by survival status: select those couples, in which only one of spouses survived or none of two. Built survival statistics by class, i.e. ratio of couples with partial survival divided by total number of couples in class.

Input is indexed with `PassengerId` and is a concatenation of train and test sets. Output must be `Series` with three elements indexed by values from `Pclass` column.

In [ ]:
def find_couples(df):
    family_names = (df
                .replace(re.compile(r'\s+\(.*\)'), '')
                .replace(re.compile("Mrs."), "Mr."))[["Name", "Sex","Survived","Pclass"]]
    womens_names = family_names[(family_names.Sex=="female") & family_names.Name.str.contains("Mr.")]
    mens_names = family_names[(family_names.Sex=="male") & family_names.Name.str.contains("Mr.")]
    family = pd.merge(mens_names,womens_names, how="inner", left_on="Name", right_on="Name", suffixes=("","_f"))[["Pclass","Survived","Survived_f"]]
    family.dropna(inplace=True)
    print(family.groupby('Pclass').size())
    return family[(family['Survived'] != 1.0) | (family['Survived_f'] != 1.0)].groupby("Pclass").size() / family.groupby("Pclass").size()

In [ ]:
PROBLEM_ID = 8

if TEST:
    total_grade += solutions.check(STUDENT, PROBLEM_ID, find_couples)

In [ ]:
find_couples(data)